# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests 
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load in the data from WeatherPy
path = 'WeatherPy/city_data.csv'
vacation_df = pd.read_csv(path)
vacation_df


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Deqing,CN,30.5448,119.9599,24.44,67,100,0.58
1,Haines Junction,CA,60.7522,-137.5108,10.00,53,100,2.06
2,Butaritari,KI,3.0707,172.7902,27.96,80,100,5.99
3,Ushuaia,AR,-54.8000,-68.3000,10.00,53,75,4.63
4,Klaksvík,FO,62.2266,-6.5890,4.00,87,75,9.77
...,...,...,...,...,...,...,...,...
514,Mahibadhoo,MV,3.7833,72.9667,29.02,74,16,5.77
515,Panique,PH,12.4797,122.0153,31.05,66,97,4.75
516,Oktyabr'skoye,RU,43.0645,44.7417,14.00,100,100,2.00
517,Broken Hill,AU,-31.9500,141.4333,16.00,51,0,2.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [14]:
locations = vacation_df[['Latitude', 'Longitude']]

humidity = vacation_df['Humidity']

In [22]:
fig = gmaps.figure(center=(46.0, -5.0),zoom_level=1)
max_intensity = np.max(humidity)

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=100,
                                    point_radius=2)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Zero cloudiness
cloud_df = vacation_df[vacation_df['Cloudiness'] == 0]

temp_df = cloud_df[(cloud_df['Temperature'] > 25) & (cloud_df['Temperature'] < 35)]

wind_df = temp_df[temp_df['Wind Speed'] < 10]

humidity_df = wind_df[wind_df['Humidity'] < 45]

hotel_df = humidity_df.dropna()
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
309,Najrān,SA,17.4924,44.1277,26.00,18,0,2.26
382,Sarai Naurang,PK,32.8259,70.7814,30.25,31,0,3.09
477,Riyadh,SA,24.6877,46.7219,30.00,20,0,1.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [26]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Missing field/result... skipping.
------------
Closest hotel in Sarai Naurang is Azad Waziristan Hotel.
------------
Closest hotel in Riyadh is Four Seasons Hotel Riyadh At Kingdom Center.
------------
-------End of Search-------


In [27]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
382,Sarai Naurang,PK,32.8259,70.7814,30.25,31,0,3.09,Azad Waziristan Hotel
477,Riyadh,SA,24.6877,46.7219,30.00,20,0,1.54,Four Seasons Hotel Riyadh At Kingdom Center


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))